In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
!pip install -qU langchain jq tiktoken langchain_community langchain_chroma langchain-huggingface huggingface-hub sentence_transformers chromadb langchainhub
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Langchain

In [34]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline, HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceBgeEmbeddings, HuggingFaceHubEmbeddings, HuggingFaceHubEmbeddings
from langchain_huggingface import HuggingFacePipeline
import bs4, getpass, os, tiktoken
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from transformers import BitsAndBytesConfig
from IPython.display import display, Markdown

In [4]:


# Load, chunk and index the contents of the blog.
#loader = TextLoader("/content/output/output.md")
#docs = loader.load()

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
#num_tokens_from_string(question, "cl100k_base")
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)
#similarity = cosine_similarity(query_result, document_result)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
vectorstore = Chroma(persist_directory="/content/drive/MyDrive/UWA/Sem 4/Capstone/Project/vector1",embedding_function=hf)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:

#def format_docs(docs):
#    return "\n\n".join(" page number : "+ str(doc.metadata['page']) +" "+ doc.page_content for doc in docs)


# Function to format documents for inclusion in the context
def format_docs(docs):
    """
    Formats a list of documents into a string with content and metadata.

    Args:
        docs (list): A list of document objects with 'page_content' and 'metadata' attributes.

    Returns:
        str: A formatted string with each document's content and metadata.
    """
    return "\n\n".join(" page number : "+ str(doc.metadata['page']) +" "+ doc.page_content for doc in docs)

In [8]:

from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline, BitsAndBytesConfig

model_name = "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype="auto",device_map="cuda", trust_remote_code=True, attn_implementation="flash_attention_2")


tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [9]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500)
llm = HuggingFacePipeline(pipeline=pipe)

In [10]:
question = "What Ideal gas law?"

In [11]:
docs = retriever.get_relevant_documents(question)
total_tokens = 0
for doc in docs:
    # Extract the page content
    content = doc.page_content

    # Calculate the number of tokens in the page content
    tokens = num_tokens_from_string(content, "cl100k_base")
    total_tokens += tokens

print(total_tokens)

<ipython-input-11-66c80b1c3f05>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


916


In [12]:
docs

[Document(metadata={'page': 22}, page_content="CHAPTER 1: Definitions and Principles 2\n\n $R$ Gas law constant, 8.31447 x 10 J/K . kg mol, 8.31447 x 107\nergs/K.g mol, or 1.98588 Btu/°R . lb mol\n $T$ Bineahece wo as pn $m_{a}$ and $m_{b} \\left[ \\mathrm{E q.} \\left( 1. 2 \\right) \\right]$ \n $r$ \n $\\mathrm{K}, \\,^{\\circ} \\mathrm{C}, \\,^{\\circ} \\mathrm{R}, \\, \\mathrm{o r} \\,^{\\circ} \\mathrm{F} ;$ ; thermodynamic absolute temperature $\\bar{T}$ [Eq. (1.1l)]\nDimension of temperature\n $t$ Time, s; I, dimension of time\n $V$ Linear velocity, m/s, cm/s, or ft/s; $u_{0},$ initial velocity of falling body $u$ \nVolume, m?, cm-, or ft'\n $\\bar{V}$ Average fluid velocity, m/s, cm/s, or ft/s\n $W$ Work, J, ergs, or ft . 1b,; $W_{c},$ work delivered by system during cycle $y$  $A$ , $B$ ,and $C$ ningasmixure $y_{A}, y_{B}, y_{C},$ mole fractions of component:\n\nZ Height above datum plane, m, cm, or ft\n\nCompressibility factor, dimensionless\n\n## Greek Letters\n\n $\\triangl

In [23]:
if total_tokens < 1000:
    template = """
    <|system|>
    Answer the question and all the page numbers where this information is found based in the information provided in the context.
    Providing all the relevant page numbers is essential.

    Context: {context}
    <|end|>

    <|user|>
    Question: {question}
    <|end|>
    <|assistant|>

    """

else:
    template = """Do not answer the question; only provide the page numbers where they can find the information.

    Context: {context}
    Question: {question}

    """

prompt = ChatPromptTemplate.from_template(template)

In [24]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

def extract_assistant_response(answer):
    if "<|assistant|>" in answer:
        return answer.split("<|assistant|>")[1].strip()
    return answer

# Generate the answer
raw_answer = rag_chain.invoke(question)
cleaned_answer = extract_assistant_response(raw_answer)


In [25]:
cleaned_answer

'The Ideal gas law is given by the equation:\n\n$$\nz = \\frac{pV}{nRT} = 1\n$$\n\nwhere:\n- $z$ is the compressibility factor (dimensionless)\n- $p$ is the pressure of the gas (in units of force per unit area, such as Pascals or atmospheres)\n- $V$ is the volume of the gas (in units of length, such as cubic meters)\n- $n$ is the number of moles of the gas\n- $R$ is the gas constant (8.31447 x 10^7 J/K·mol, 8.31447 x 10^7 erg/K·mol, or 1.98588 Btu/°R·lb mol)\n- $T$ is the absolute temperature (in Kelvin, Celsius, Rankine, or Fahrenheit)\n\nIn the context provided, the Ideal gas law is mentioned on page 18, and it is also referred to as Eq. (1.56). The equation shows that for an ideal gas, the ratio of pressure and volume is directly proportional to the temperature, and the compressibility factor ($z$) is equal to 1. This law is useful for practical calculations within a range of densities and is a simplified version of the more complex real gas behavior.'

In [35]:
class RAGChatModel:
    def __init__(self, retriever, llm, tokenizer, max_token_limit=1000):
        self.retriever = retriever
        self.llm = llm
        self.tokenizer = tokenizer
        self.max_token_limit = max_token_limit
        self.current_token_count = 0
        self.template_standard =  """
        <|system|>
        Answer the question and all the page numbers where this information is found based in the information provided in the context.
        Providing all the relevant page numbers is essential.
        Context: {context}
        <|end|>
        <|user|>
        Question: {question}
        <|end|>
        <|assistant|>
        """
        self.template_exceeded = """
        <|system|>
        Do not answer the question; only provide the page numbers where they can find the information. the responese should only contain page numbers
        Context: {context}
        <|end|>
        <|user|>
        Question: {question}
        <|end|>
        <|assistant|>
        """

    def num_tokens_from_string(self, string: str) -> int:
        """Returns the number of tokens in a text string using a specific tokenizer."""
        return len(self.tokenizer.encode(string))

    def format_docs(self, docs):
        """Format the documents to be used as context in the prompt."""
        return "\n\n".join(f"Page number: {doc.metadata['page']}\n{doc.page_content}" for doc in docs)

    def get_prompt(self, docs, question):
        """Choose the appropriate template based on the token count."""
        # Format the context
        context = self.format_docs(docs)
        total_tokens_in_context = self.num_tokens_from_string(context)

        # Add tokens to the running total
        self.current_token_count += total_tokens_in_context

        # Decide which template to use
        if self.current_token_count > self.max_token_limit:
            print("Token limit exceeded. Switching to shortened template.")
            template = self.template_exceeded
        else:
            template = self.template_standard

        # Create the prompt
        prompt = template.format(context=context, question=question)
        return prompt

    def extract_clean_answer(self, raw_output):
        """Extract only the answer from the raw output."""
        # Find the start of the assistant's answer (after "<|assistant|>")
        assistant_tag = "<|assistant|>"
        if assistant_tag in raw_output:
            # Split the output and take everything after "<|assistant|>"
            clean_answer = raw_output.split(assistant_tag)[-1].strip()
            return clean_answer
        return raw_output.strip()  # If tag not found, return the raw output

    def ask_question(self, question):
        """Main function to retrieve relevant docs and generate a response."""
        # Retrieve relevant documents
        docs = self.retriever.get_relevant_documents(question)

        # Generate prompt based on current token count
        prompt = self.get_prompt(docs, question)

        # Pass the prompt as a list to the LLM
        result = self.llm.generate([prompt])  # Passing prompt as a list

        # Extract the generated text from the LLMResult object
        raw_answer = result.generations[0][0].text

        # Post-process the output to get the clean answer
        clean_answer = self.extract_clean_answer(raw_answer)

        # Display the cleaned answer as markdown
        display(Markdown(clean_answer))

# Initialize the RAGChatModel with the retriever, LLM, and tokenizer.
rag_chat_model = RAGChatModel(retriever, llm, tokenizer)

# Simulate an interactive chat
print("Welcome to the RAG Chat Model! Ask any question (type 'exit' to quit):")
while True:
    # Get user input
    question = input("Your question: ")

    # Exit condition
    if question.lower() == 'exit':
        print("Exiting the chat.")
        break

    # Generate and display the response in markdown
    rag_chat_model.ask_question(question)

Welcome to the RAG Chat Model! Ask any question (type 'exit' to quit):
Your question: what is ideal gas law
Token limit exceeded. Switching to shortened template.


The ideal gas law can be found on page 18 of the provided context. The ideal gas law is given by the equation:

$$
z={\frac{p V} {n R T}}={\frac{p} {\rho_{M} R T}}=1 \qquad\qquad\qquad\qquad( 1. 5 6 )
$$

This equation relates the pressure (p), volume (V), amount of substance in moles (n), the ideal gas constant (R), and the absolute temperature (T). It assumes that the gas behaves ideally, meaning that the gas particles do not interact with each other and occupy no volume themselves. The ideal gas law is a good approximation for many gases under a wide range of conditions, although it becomes less accurate at very high pressures and low temperatures.

KeyboardInterrupt: Interrupted by user